# Parameter-Shift法をQiskitで実装する

In [1]:
from qiskit import Aer
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import I,X, Y, Z, StateFn, CircuitStateFn
from qiskit.aqua.operators.expectations import PauliExpectation, AerPauliExpectation
from qiskit.aqua.operators.converters import CircuitSampler
from qiskit.aqua.operators.gradients import Gradient
import numpy as np
import matplotlib as plt
%matplotlib inline

C:\Users\DaikiMurata\anaconda3\lib\site-packages\qiskit\aqua\operators\operator_globals.py:48: DeprecationWarning: `from_label` is deprecated and will be removed no earlier than 3 months after the release date. Use Pauli(label) instead.
  X = make_immutable(PrimitiveOp(Pauli.from_label('X')))


題材は[QHACK2021で使用された問題](https://github.com/XanaduAI/QHack2021/blob/main/QML_Challenges/quantum_gradients_100_template/problem.pdf)を使用します。

入力パラメータ  
```
[1, 0.5, -0.765, 0.1, 0, -0.654]
```

期待される出力  
```
[0, 0, 0, 0, -0.4553474723, 0]
```

## モジュールを利用せずにゼロから実装する

In [2]:
# バックエンドや量子インスタンスを定義
backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

ハミルトニアンの期待値を求める関数を定義します。

In [3]:
def exp_val(params):
    n = 3
    qc = QuantumCircuit(n)
    param_list = ParameterVector('Parameter', 2*n)

    for i in range(len(param_list)//n):
        qc.rx(param_list[3*i], 0)
        qc.ry(param_list[3*i+1], 1)
        qc.rz(param_list[3*i+2], 2)

        qc.cnot(0, 1)
        qc.cnot(1, 2)
        qc.cnot(2, 0)

    param_dict = dict(zip(param_list.params, params))
    qc.assign_parameters(param_dict, inplace=True)

    op = Z ^ I ^ Y # ハミルトニアンを定義
    psi = CircuitStateFn(qc) # 状態ベクトルを定義
    measurable_expression = StateFn(op, is_measurement=True).compose(psi) 

    # 期待値を計算
    expectation = AerPauliExpectation().convert(measurable_expression)

    sampler = CircuitSampler(q_instance).convert(expectation) 
    return sampler.eval().real

In [4]:
params = [1, 0.5, -0.765, 0.1, 0, -0.654]

In [5]:
gradient = np.zeros_like(params)
for i in range(len(params)):
    shifted = params.copy()
    shifted[i] += np.pi/2
    forward = exp_val(shifted)

    shifted[i] -= np.pi
    backward = exp_val(shifted)

    gradient[i] = 0.5 * (forward - backward)

C:\Users\DaikiMurata\anaconda3\lib\site-packages\qiskit\quantum_info\operators\symplectic\pauli.py:185: DeprecationWarning: Initializing Pauli from `Pauli(z=z, x=x)` kwargs is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use tuple initialization `Pauli((z, x))` instead.
  base_z, base_x, base_phase = self._from_array_deprecated(z, x)
C:\Users\DaikiMurata\anaconda3\lib\site-packages\qiskit\aqua\operators\primitive_ops\pauli_op.py:93: DeprecationWarning: `kron` is deprecated and will be removed no earlier than 3 months after the release date of Qiskit Terra 0.17.0. Use `expand` instead, but note this does not change the operator in-place.
  return PauliOp(op_copy.kron(self.primitive), coeff=self.coeff * other.coeff)


In [6]:
print(np.round(gradient, 10))

[ 0.          0.          0.          0.         -0.45534747  0.        ]


## Using Qiskit Gradient Framework


In [7]:
params = [1, 0.5, -0.765, 0.1, 0, -0.654]

In [8]:
def parameter_shift_gradient(params):
    n = 3
    qc = QuantumCircuit(n)
    param_list = ParameterVector('param_list', 2*n)

    for i in range(len(param_list)//n):
        qc.rx(param_list[3*i], 0)
        qc.ry(param_list[3*i+1], 1)
        qc.rz(param_list[3*i+2], 2)

        qc.cnot(0, 1)
        qc.cnot(1, 2)
        qc.cnot(2, 0)

    param_dict = dict(zip(param_list.params, params))

    obs = Z ^ I ^ Y
    psi = CircuitStateFn(qc)
    operator = ~StateFn(obs) @ psi

    state_grad = Gradient(grad_method="param_shift").convert(operator=operator, params=param_list)
    result = state_grad.assign_parameters(param_dict).eval()
    return np.array(result).real

In [9]:
print(np.round(parameter_shift_gradient(params), 10))

[ 0.          0.          0.          0.         -0.45534747  0.        ]


## バージョン情報

In [10]:
import qiskit.tools.jupyter
%qiskit_version_table